### Load Data

In [1]:
import os
os.listdir()

['pollution.ipynb',
 'Untitled.ipynb',
 'main_flow',
 'testasync.ipynb',
 '.Trash-0',
 'testdatetime.ipynb',
 '.ipynb_checkpoints',
 'test_pollution.ipynb',
 'test.ipynb']

In [1]:
# select ชื่อเมืองในไทย
import os
import pandas as pd

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  # เดินขึ้น 1 ระดับจาก /home/jovyan/work
print(BASE_DIR)
coord_path = os.path.join(BASE_DIR, "save", "amphoe_coord.csv")
print(coord_path)

coord_df = pd.read_csv(coord_path)
coord_df


/home/jovyan
/home/jovyan/save/amphoe_coord.csv


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/save/amphoe_coord.csv'

In [22]:
#test
df_sample = coord_df.sample(n=10)
df_sample

,province_id,province,provinceEN,amphoe_id,amphoe,amphoeEN,lat,lon
207,63,ตาก,Tak,6301,เมืองตาก,Mueang Tak,16.869680,99.128980
458,66,พิจิตร,Phichit,6602,วังทรายพูน,Wang Sai Phun,16.339800,100.552100
246,30,นครราชสีมา,Nakhon Ratchasima,3025,วังน้ำเขียว,Wang Nam Khiao,14.470889,101.824609
409,14,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,1413,บางซ้าย,Bang Sai,14.306400,100.293700
404,94,ปัตตานี,Pattani,9402,โคกโพธิ์,Khok Pho,6.717000,101.100000
52,10,กรุงเทพมหานคร,Bangkok,1018,คลองสาน,Khlong San,13.728700,100.501000
602,33,ศรีสะเกษ,Si Saket,3304,กันทรลักษ์,Kantharalak,14.640000,104.650000
583,51,ลำพูน,Lamphun,5103,บ้านโฮ่ง,Ban Hong,18.331111,98.818889
270,80,นครศรีธรรมราช,Nakhon Si Thammarat,8015,ขนอม,Khanom,9.204722,99.861111
924,58,แม่ฮ่องสอน,Mae Hong Son,5807,ปางมะผ้า,Pang Mapha,19.522500,98.246111


### Fetch Data

In [8]:
# Test Find Lat Lon
# province = "Pathum Thani"
# print(
#     provinces[province]['lat'],
#     provinces[province]['lon']
# )
# coord_df

,province_id,province,provinceEN,amphoe_id,amphoe,amphoeEN,lat,lon
0,81,กระบี่,Pathum Thani,8108,เหนือคลอง,Nuea Khlong,8.074722,99.003611
1,81,กระบี่,Pathum Thani,8107,ลำทับ,Lam Thap,8.071667,99.291667
2,81,กระบี่,Pathum Thani,8106,ปลายพระยา,Plai Phraya,8.533333,98.862778
3,81,กระบี่,Pathum Thani,8102,เขาพนม,Khao Phanom,8.264722,99.049167
4,81,กระบี่,Pathum Thani,8105,อ่าวลึก,Ao Luek,8.377500,98.721667
...,...,...,...,...,...,...,...,...
920,58,แม่ฮ่องสอน,Pathum Thani,5803,ปาย,Pai,19.358611,98.440000
921,58,แม่ฮ่องสอน,Pathum Thani,5804,แม่สะเรียง,Mae Sariang,18.159444,97.933611
922,58,แม่ฮ่องสอน,Pathum Thani,5805,แม่ลาน้อย,Mae La Noi,18.384444,97.936944
923,58,แม่ฮ่องสอน,Pathum Thani,5806,สบเมย,Sop Moei,17.961944,97.933333


In [23]:
import requests
import pandas as pd
from datetime import datetime
import time
import pytz
from prefect import flow, task # Prefect flow and task decorators

def get_pollution_data(context):
    lat = context['lat']
    lon = context['lon']
    province = context['province']
    amphoe = context['amphoe']

    API_KEY = "68c16da5e675ad8635df84629765b118"
    # WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    POLLUTION_ENDPOINT = "http://api.openweathermap.org/data/2.5/air_pollution"
    # province=province_context['province']
    
    params = {
        # "lat": province_context['lat'],
        # "lon": province_context['lon'],
        "lat" : lat,
        "lon" : lon,
        "appid": API_KEY,
        "units": "metric"
    }
    
    try:
        # Make API request
        response = requests.get(POLLUTION_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        # return data
        
        # Convert timestamp to datetime       
        dt = datetime.utcnow()
        thai_tz = pytz.timezone('Asia/Bangkok')
        localtime = dt.astimezone(thai_tz)
        
        # Create dictionary with required fields
        components = data['list'][0]['components']
        pollution_dict = {
            'timestamp': dt,
            'year': dt.year,
            'month': dt.month,
            'day': dt.day,
            'hour': dt.hour,
            'minute': dt.minute,
            'localtime': localtime,
            'province' : province,
            'amphoe' : amphoe,
            'lat' : data['coord']['lat'],
            'lon' : data['coord']['lon'],
            # 'location': data['name'],
            'aqi' : data['list'][0]['main']['aqi'],
            'co' : components['co'],
            'no' : components['no'],
            'no2' : components['no2'],
            'o3' : components['o3'],
            'so2' : components['so2'],
            'pm25' : components['pm2_5'],
            'pm10' : components['pm10'],
            'nh3' : components['nh3']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return pollution_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None
    except Exception as e:
        print(f"Error: {e} at {province} - {amphoe}")
        return None

# get_pollution_data()

In [24]:
# def create_df():
# สมมติคุณมี coord_df อยู่แล้ว

import time

start_time = time.time()

pollution_data = []
    
for _, row in df_sample.iterrows():
        context = {
            'province': row['provinceEN'],
            'amphoe': row['amphoeEN'],
            'lat': row['lat'],
            'lon': row['lon']
        }
        result = get_pollution_data(context)
        if result:
            pollution_data.append(result)
        
        # time.sleep(1)  # พัก API เพื่อไม่ให้โดน block
    
    #สร้าง dataframe
pollution_df = pd.DataFrame(pollution_data)
# pollution_df(head)
end_time = time.time()
elapsed_time = end_time - start_time
print(f"ใช้เวลาในการรันทั้งหมด: {elapsed_time:.2f} วินาที")
# create_df()
pollution_df

Error fetching data: 429 Client Error: Too Many Requests for url: http://api.openweathermap.org/data/2.5/air_pollution?lat=16.86968&lon=99.12898&appid=68c16da5e675ad8635df84629765b118&units=metric
Error fetching data: 429 Client Error: Too Many Requests for url: http://api.openweathermap.org/data/2.5/air_pollution?lat=16.3398&lon=100.5521&appid=68c16da5e675ad8635df84629765b118&units=metric
Error fetching data: 429 Client Error: Too Many Requests for url: http://api.openweathermap.org/data/2.5/air_pollution?lat=14.4708894&lon=101.8246093&appid=68c16da5e675ad8635df84629765b118&units=metric
Error fetching data: 429 Client Error: Too Many Requests for url: http://api.openweathermap.org/data/2.5/air_pollution?lat=14.3064&lon=100.2937&appid=68c16da5e675ad8635df84629765b118&units=metric
Error fetching data: 429 Client Error: Too Many Requests for url: http://api.openweathermap.org/data/2.5/air_pollution?lat=6.717&lon=101.1&appid=68c16da5e675ad8635df84629765b118&units=metric
Error fetching dat

""


### FLOW

In [ ]:
# @flow(name="main-flow", log_prints=True)
def main_flow():
  
    # df=pd.DataFrame([get_pollution_data(
    #     {
    #         'province':province,
    #         'lat':provinces[province]['lat'],
    #         'lon':provinces[province]['lon'],
    #     }
    # ) for province in list(provinces.keys())])

# สมมติคุณมี coord_df อยู่แล้ว
    pollution_data = []
    
    for _, row in coord_df.iterrows():
        context = {
            'province': row['Province'],
            'amphoe': row['Amphoe'],
            'lat': row['Latitude'],
            'lon': row['Longtitude']
        }
        result = get_pollution_data(context)
        if result:
            pollution_data.append(result)
        
        time.sleep(1)  # พัก API เพื่อไม่ให้โดน block
    
    #สร้าง dataframe
    pollution_df = pd.DataFrame(pollution_data)
    
    # lakeFS credentials from your docker-compose.yml
    ACCESS_KEY = "access_key"
    SECRET_KEY = "secret_key"
    
    # lakeFS endpoint (running locally)
    lakefs_endpoint = "http://lakefs-dev:8000/"
    
    # lakeFS repository, branch, and file path
    repo = "pollution-data"
    branch = "main"
    path = "pollution.parquet"
    
    # Construct the full lakeFS S3-compatible path
    lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"
    
    # Configure storage_options for lakeFS (S3-compatible)
    storage_options = {
        "key": ACCESS_KEY,
        "secret": SECRET_KEY,
        "client_kwargs": {
            "endpoint_url": lakefs_endpoint
        }
    }
    df.to_parquet(
        lakefs_s3_path,
        storage_options=storage_options,
        partition_cols=['year','month','day','hour'],
        
    )
main_flow()

In [16]:
 # lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"
    
# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"
    
    # lakeFS repository, branch, and file path
repo = "pollution-data"
branch = "main"
path = "pollution.parquet"
    
    # Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"
    
    # Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
    }
pollution_df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour'],
        
)

In [ ]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head()

In [ ]:
#test read parquet
path_all_partition = 's3a://weather/main/weather.parquet/'
path_single_partition = 's3a://weather/main/weather.parquet/year=2025/month=4/day=18/hour=13/fcf052bcde4e496d89f6685cad14f951-0.parquet'
df2 = pd.read_parquet(
    path=path_all_partition,
    storage_options=storage_options,
)
df2.info()
df2.head()